In [1]:
import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 3s 0us/step


In [2]:
base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [3]:
model = keras.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_freeze = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=128)

Epoch 1/5
391/391 [==============================] - 744s 2s/step - loss: 1.5694 - accuracy: 0.4505 - val_loss: 1.3189 - val_accuracy: 0.5368
Epoch 2/5
391/391 [==============================] - 737s 2s/step - loss: 1.3240 - accuracy: 0.5360 - val_loss: 1.2463 - val_accuracy: 0.5598
Epoch 3/5
391/391 [==============================] - 735s 2s/step - loss: 1.2596 - accuracy: 0.5586 - val_loss: 1.2057 - val_accuracy: 0.5769
Epoch 4/5
391/391 [==============================] - 689s 2s/step - loss: 1.2186 - accuracy: 0.5755 - val_loss: 1.1799 - val_accuracy: 0.5854
Epoch 5/5
391/391 [==============================] - 720s 2s/step - loss: 1.1901 - accuracy: 0.5856 - val_loss: 1.1586 - val_accuracy: 0.5919


In [ ]:
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_unfreeze = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=128)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_freeze.history['accuracy'])
plt.plot(history_freeze.history['val_accuracy'])
plt.plot(history_unfreeze.history['accuracy'])
plt.plot(history_unfreeze.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train (Freeze)', 'Val (Freeze)', 'Train (Unfreeze)', 'Val (Unfreeze)'], loc='lower right')
plt.show()